# Creating Vector Database 

In [2]:
import chromadb

client = chromadb.PersistentClient(path="C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/vectorDB")

ModuleNotFoundError: No module named 'chromadb'